### SageMaker  Endpoint 部署ChatGLM
  
[ChatGLM](https://github.com/THUDM/ChatGLM-6B): ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 General Language Model (GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。 ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备inference.py, requirements.txt
3. 准备s5cmd

In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [4]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


In [12]:
!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'chatglm')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'chatglm')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

dummy
upload: ./model.tar.gz to s3://sagemaker-us-west-2-687912291502/chatglm/assets/model.tar.gz


In [ ]:
!mkdir -p code
!cp -r ./inference/  ./code/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ./code/

In [2]:
#!aws s3 cp s3://sagemaker-us-west-2-687912291502/huggingface-chatglm-simple-2023-05-06-1-2023-05-06-14-24-12-728/output/model.tar.gz ./
#!tar -xvf model.tar.gz
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/simple/adgen-chatglm-6b-ft/
#!chmod +x ./code/s5cmd && ./code/s5cmd sync s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed/adgen-chatglm-6b-ft/* ./tmp2/
!rm -rf ./tmp2

### local test only

In [ ]:
import os
import json
import uuid
import io
import sys

import traceback

from PIL import Image

import requests
import boto3
import sagemaker
import torch


from torch import autocast
#from transformers import T5Tokenizer, T5ForConditionalGeneration
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

def preprocess(text):
    text = text.replace("\n", "\\n").replace("\t", "\\t")
    return text

def postprocess(text):
    return text.replace("\\n", "\n").replace("\\t", "\t")

def answer(text, sample=True, top_p=0.45, temperature=0.7,model=None):
    text = preprocess(text)
    response, history = model.chat(tokenizer, text, history=[])
    
    return postprocess(response)


def model_fn(model_dir):
    print("=================model_fn_Start=================")
    model_s3_path = os.environ['MODEL_S3_PATH']
    print("=================model s3 path=================="+model_s3_path)
    os.system("./s5cmd sync {0} {1}".format(model_s3_path+"*","./tmp/model/"))
    if os.environ["MODEL_TYPE"] == "ptuning":
        config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, pre_seq_len=128)
        model = AutoModel.from_pretrained("THUDM/chatglm-6b", config=config, trust_remote_code=True)
        prefix_state_dict = torch.load(os.path.join("./tmp/model/", "pytorch_model.bin"))
        new_prefix_state_dict = {}
        for k, v in prefix_state_dict.items():
            if k.startswith("transformer.prefix_encoder."):
                new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
        model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

    elif os.environ["MODEL_TYPE"] == "full turning":
        print("====================load full turning=================") 
        config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, pre_seq_len=128)
        model = AutoModel.from_pretrained("./tmp/model/",config=config, trust_remote_code=True)
    else:
        print("====================load normal ======================")
        config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, pre_seq_len=128)
        model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).half().cuda()

    #model = model.to("cuda")
    model = model.quantize(4)
    model = model.half().cuda()
    model.transformer.prefix_encoder.float()
    model = model.eval()
    print("=================model_fn_End=================")
    return model

model = model_fn("s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed/adgen-chatglm-6b-ft/")
result=answer(input_data['ask'], model=model)

### 设置模型推理参数
1. model_type : p-turning or full tuning
2. model_s3_path: fine tuned model s3 url

In [73]:
model_name = "chatglm-inference"
entry_point = 'inference-chatglm.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
    #'MODEL_TYPE'                    : 'ptuning',
    #'MODEL_TYPE'                    : 'full turning',
    'MODEL_TYPE'                    : 'normal',
    #'MODEL_S3_PATH'                 : 's3://sagemaker-us-west-2-687912291502/llm/models/chatglm/simple/adgen-chatglm-6b-ft/'
    'MODEL_S3_PATH'                 : 's3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed-mutip-nodes/adgen-chatglm-6b-ft/'
}

In [74]:
from sagemaker.pytorch.model import PyTorchModel
#model_data = "s3://sagemaker-us-west-2-687912291502/huggingface-chatglm-simple-2023-05-06-1-2023-05-06-14-24-12-728/output/model.tar.gz"

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

### 部署 SageMaker Endpoint
部署推理服务

In [75]:
endpoint_name = None
instance_type = 'ml.g5.8xlarge'
instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

-----------!

### ChatGLM 测试


In [76]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [79]:

inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好！请问有什么需要帮助的吗？
以下建议有助于改善晚上睡不着的情况：

1. 保持规律的睡眠时间：尽量在同一时间上床睡觉，建立规律的睡眠时间，让身体适应固定的睡眠时间。

2. 创造舒适的睡眠环境：保持卧室安静、凉爽、黑暗、舒适，使用舒适的床垫和枕头，确保睡眠环境符合个人需求。

3. 放松身心：在睡前进行放松活动，如冥想、深呼吸、温水泡脚、听轻柔的音乐等，放松


In [80]:
inputs= {
    "ask": "列出一些年夜饭好意头的菜肴以及其寓意!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一篇人工智能课程的教案，1000字"

}

response = predictor.predict(inputs)
print(response["answer"])

以下是一些年夜饭好意头的菜肴以及其寓意：

1. 鱼：年年有余，表示希望家中财源广进。在菜肴中以鲤鱼最常见，但也有人用鲈鱼、鲳鱼等鱼类制作。

2. 年糕：年年高升，寓意着新年里生活越来越好，前程似锦。

3. 香菇发菜：发财发福，吉祥如意，是象征财源广进的重要食材
人工智能课程的教案示例如下：

主题：人工智能的基础

目标：让学生了解人工智能的基本概念、原理和应用领域，掌握使用Python编程语言进行人工智能开发的基本技能。

教学内容：

1. 人工智能的基本概念

- 什么是人工智能
- 人工智能的应用领域
- 人工智能的特点

2. Python编程语言

- Python编程语言的基本语法
-


In [90]:
#inputs= {
#    "ask": "怎么修改huggingface transformers的model cache位置"
#
#}
#
#response = predictor.predict(inputs)
#print(response["answer"])

inputs= {
    "ask": "民法典第一百零二条"
    #"ask": "我租的房子被房东卖了，现在要把我赶出去，我怎么办？"

}

response = predictor.predict(inputs)
print(response["answer"])

民法典第一百零二条 当事人可以依据本法典第一百二十七条、第一百二十八条的规定条的规定，就特定民事关系达成书面协议。

当事人达成的书面协议，符合本法典第一百二十七条或者第一百二十八条的规定的，应当作为确定民事关系的有效协议。

当事人未达成书面协议，或者达成的协议不符合本法典第一百二十七条或者第一百二十八条的规定的，可以按照本


In [82]:
#我们来查看一下刚部署的这个ChatGLM模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

'chatglm-inference-2023-05-09-10-08-10-446'

利用已经在SageMaker real time inference endpoint部署的ChatGLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [83]:
import json
import boto3

client = boto3.client('runtime.sagemaker')

def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


In [84]:
sagemaker_endpoint_name

'chatglm-inference-2023-05-09-10-08-10-446'

先简单测试一下ChatGLM针对单个问题的回答

In [85]:
#payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}

In [86]:
payload1 = {"ask": """问题:GPU优化作用？
                      回答:一般来说，GPU无法在检索数据同时执行这些计算。此外，现代GPU的计算性能远远高于每个操作(被称为GPU编程中的核)所需的内存传输速度。
核融合是一种基于GPU计算的优化方法，通过在一次内核调用中执行多个连续操作。该方法提供了一种最小化数据传输的方法：中间结果留在GPU寄存器中，而不是复制到VRAM，从而节省开销。
我们使用了Megatron-LM提供了几个定制化融合CUDA核。首先，我们使用一个优化核来执行LayerNorm，以及用核来融合各种缩放、掩码和softmax操作的各种组合。
使用Pytorch的JIT功能将一个偏差项添加至GeLU激活中。作为一个使用融合核的例子，在GeLU操作中添加偏差项不会增加额外的时间，因为该操作受内存限制：与GPU VRAM和寄存器之间的数据传输相比，额外的计算可以忽略不计。
因此融合这两个操作基本上减少了它们的运行时间
                     问题：主题摘要,限制在10个字内
           """}

payload2 ={ "ask": """问题：如何优化BLOOM？
                      回答：我们使用上表3中详细描述的超参数来训练BLOOM的6个尺寸变体。
架构和超参数来自于我们的实验结果(Le Scao et al.)和先前的训练大语言模型(Brown et al.)。
非176B模型的深度和宽度大致遵循先前的文献(Brown et al.)，偏离的3B和7.1B只是为了更容易适合我们训练设置。
由于更大的多语言词表，BLOOM的embedding参数尺寸更大。在开发104B参数模型的过程中，我们使用了不同的Adam 
参数、权重衰减和梯度裁剪来对目标稳定性进行实验，但没有发现其有帮助。
对于所有模型，我们在410B tokens使用cosine学习率衰减调度，在计算允许的情况下，将其作为训练长度的上限，并对375M tokens进行warmup。
我们使用权重衰减、梯度裁剪，不使用dropout。ROOTS数据集包含341B tokens的文本。然而，基于训练期间发布的修订scaling laws，我们决定在重复数据上对大模型进行额外25B tokens的训练。
由于warmup tokens + decay tokens大于总的token数量，所以学习率衰减始终未达到终点
问题：主题摘要,限制在20个字内
"""}
query_response = query_endpoint_with_json_payload(json.dumps(payload2).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

BL


In [87]:
#payload={"ask":"""好累啊，想去米亚罗看红叶
#问题：内容分类,以上属于闲聊还是专业问题？"""}

payload={"ask":"""新加坡有什么好玩的么？
问题：内容分类,以上是闲聊还是提问专业问题？"""}

#payload={"ask":"""AWS是什么公司？
#问题：内容分类,以上是闲聊还是提问专业问题？"""}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

以上属于闲聊问题。


## 单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

## 多轮对话模拟：我们这里来测试一下ChatGLM的多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导ChatGLM，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nChatGLM：您好!我是ChatGLM。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉ChatGLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "ChatGLM：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()